In [1]:
import os
import subprocess
from pydub import AudioSegment
import pandas as pd
import base64
from google import genai
from google.genai import types
from tqdm import tqdm
from pathlib import Path

In [2]:


def chop_and_execute(mp3_file, segment_duration=1.49, model = 'gemini-2.5-pro-preview-03-25', command="echo 'Processing temp.mp3'"):
    """
    Chops an MP3 file into segments and executes a command on each segment.

    Args:
        mp3_file (str): Path to the input MP3 file.
        segment_duration (int): Duration of each segment in seconds.
        command (str): Command to execute on each segment (e.g., "python my_script.py").
    """

    try:
        audio = AudioSegment.from_mp3(mp3_file)
        total_duration_ms = len(audio)
        segment_duration_ms = segment_duration * 1000

        onsets           = []
        offsets          = []
        laughter_dummies = []

        for start_ms in tqdm(range(0, int(total_duration_ms), int(segment_duration_ms))):
            
            end_ms = min(start_ms + segment_duration_ms, total_duration_ms)
            segment = audio[start_ms:end_ms]

            temp_file = "/Users/schmaelz/Desktop/audio/temp" + str(start_ms) + ".mp3"
            segment.export(temp_file, format="mp3")

            #print(f"Processing segment {start_ms/1000}-{end_ms/1000}s")
            onsets.append(start_ms)
            offsets.append(end_ms)
            
            yes_no_laughter = analyze_gemini(temp_file, model)
            laughter_dummies.append(yes_no_laughter)
            #os.remove(temp_file)
            

        df            = pd.DataFrame()
        df['onsets']  = onsets
        df['minutes_seconds'] = seconds_to_minutes_seconds(df['onsets'])
        df['offsets'] = offsets
        df['ls']      = laughter_dummies
        df['ls_binary'] = [1 if item.lower() == 'yes' else 0 for item in laughter_dummies]
        
        
    except FileNotFoundError:
        print(f"Error: MP3 file not found: {mp3_file}")

    return df
        

def analyze_gemini(audio_input_file_path, model):
    """
    Analyzes audio using the Gemini API.

    Args:
        audio_input_file_path: Path to the audio file.

    Returns:
        The analysis result (yes/no) as a string, or None if an error occurs.
    """
    try:
        client = genai.Client(api_key=os.environ.get("GOOGLE_API_KEY"))
        #model = "gemini-2.5-pro-preview-03-25"
        #model = my_model #"gemini-2.0-flash"
        files = [client.files.upload(file=audio_input_file_path)]
        contents = [
            types.Content(
                role="user",
                parts=[
                    types.Part.from_uri(file_uri=files[0].uri, mime_type=files[0].mime_type),
                    types.Part.from_text(
                        text="""Analyze whether this file contains clearly audible audience laughter as is typical of sitcoms. If you can identify such laughter, answer yes, otherwise answer no. Be conservative, only mark explicit cases where it is very clear that people are laughing. Only answer yes or no in lowercase letters, no other answer text or commentary. """
                    ),
                ],
            ),
        ]

        generate_content_config = types.GenerateContentConfig(response_mime_type="text/plain")

        for chunk in client.models.generate_content_stream(
            model=model, contents=contents, config=generate_content_config,
        ):
            return chunk.text

    except Exception as e:
        print(f"Error analyzing audio with Gemini: {e}")
        return 'Error'  # Indicate failure


def seconds_to_minutes_seconds(seconds_series):
    """
    Converts a pandas Series of seconds into a Series of [minutes:seconds] strings.
    Args: seconds_series (pd.Series): A pandas Series containing seconds (numeric).
    Returns:  pd.Series: A pandas Series of strings in the format [minutes:seconds].
    """
    seconds_series = seconds_series//1000
    minutes = seconds_series // 60
    remaining_seconds = seconds_series % 60
    return minutes.astype(str) + ':' + remaining_seconds.astype(str).str.zfill(2)


def find_mp3_files(folder_path):
    """
    Finds all paths to MP3 files in a folder and its subfolders.

    Args:
        folder_path (str): Path to the folder to search.

    Returns:
        list: A list of absolute paths to all MP3 files found.
    """
    mp3_files = []
    folder = Path(folder_path)

    if not folder.exists():
        print(f"Error: Folder '{folder_path}' does not exist.")
        return []

    for file_path in folder.rglob("*.mp3"):
        mp3_files.append(str(file_path.resolve()))

    return mp3_files


In [3]:
folder_to_search = "/Volumes/T7 Shield/friends/stimuli/s1" #replace with your folder path.
mp3_file_paths = find_mp3_files(folder_to_search)
mp3_file_paths.sort()
print(len(mp3_file_paths))
mp3_file_paths[:3]

48


['/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e01a.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e01b.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e02a.mp3']

In [5]:
for current_file in tqdm(mp3_file_paths[1:2]):

    current_model = 'gemini-2.5-pro-preview-03-25'
    current_outname = current_file[:-4] + '_laughter_1.49sec_' + current_model + '.csv'

    #if os.path.isfile(current_outname):
    #    # File exists
    #    print(f"The file '{current_outname}' exists. Skipping")
    #else:
        # File does not exist
    print(f"The file '{current_outname}' does not exist.")
    df2 = chop_and_execute(current_file, segment_duration=1.49, model = current_model) 
    df2.to_csv(current_outname)
        #break


  0%|                                                                                                                                       | 0/1 [00:00<?, ?it/s]

The file '/Volumes/T7 Shield/friends/stimuli/s1/friends_s01e01b_laughter_1.49sec_gemini-2.5-pro-preview-03-25.csv' does not exist.



  0%|▏                                                                                                                            | 1/591 [00:00<07:27,  1.32it/s]

Error analyzing audio with Gemini: 429 Too Many Requests. {'message': 'Response not read', 'status': 'Too Many Requests'}



  0%|▍                                                                                                                            | 2/591 [00:01<07:54,  1.24it/s]

...

Error analyzing audio with Gemini: 429 Too Many Requests. {'message': 'Response not read', 'status': 'Too Many Requests'}



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 590/591 [08:51<00:00,  1.22it/s]

Error analyzing audio with Gemini: 429 Too Many Requests. {'message': 'Response not read', 'status': 'Too Many Requests'}



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [08:53<00:00, 533.40s/it]

Error analyzing audio with Gemini: 429 Too Many Requests. {'message': 'Response not read', 'status': 'Too Many Requests'}


In [14]:
folder_to_search = "/Volumes/T7 Shield/friends/stimuli/s2" #replace with your folder path.
mp3_file_paths = find_mp3_files(folder_to_search)
mp3_file_paths.sort()
print(len(mp3_file_paths))
mp3_file_paths[:3]

48


['/Volumes/T7 Shield/friends/stimuli/s2/friends_s02e01a.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s2/friends_s02e01b.mp3',
 '/Volumes/T7 Shield/friends/stimuli/s2/friends_s02e02a.mp3']

In [25]:
mp3_file_paths = ['/Volumes/T7 Shield/friends/stimuli/s2/friends_s02e17a.mp3']

In [26]:
for current_file in tqdm(mp3_file_paths):
    current_outname = current_file[:-4] + '_laughter_1.49sec_gemini-2.0-flash.csv'

    if os.path.isfile(current_outname):
        # File exists
        print(f"The file '{current_outname}' exists. Skipping")
    else:
        # File does not exist
        print(f"The file '{current_outname}' does not exist.")
        df2 = chop_and_execute(current_file, segment_duration=1.49, model = 'gemini-2.0-flash') 
        df2.to_csv(current_outname)
        break


  0%|                                                     | 0/1 [00:00<?, ?it/s]

The file '/Volumes/T7 Shield/friends/stimuli/s2/friends_s02e17a_laughter_1.49sec_gemini-2.0-flash.csv' does not exist.



  0%|                                                     | 0/1 [13:25<?, ?it/s]


In [98]:
#mp3_file_path = "/Users/schmaelz/Desktop/s01.mp3"  #
#
#df = chop_and_execute(mp3_file_path, segment_duration=1.49) 
#df.to_csv('s01_e01_laughter_1.49sec.csv')



#import numpy as np

#d = np.load('/Users/schmaelz/Documents/01_GITHUB/nomcomm/GITHUB_PAPERS_WORKING/friends/data/00_raw_fmri/season01/task-s01e01a.npy')
#d.shape